In [ ]:
try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import os
import dlib
import cv2
import time
import math
import datetime
import matplotlib.pyplot as plt
import os
import pickle
import numpy as np
import scipy.io
import bz2
from random import randint

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:

if not os.path.exists(os.path.abspath('.') + 'mmod_human_face_detector.dat.bz2'):
    annotation_zip = tf.keras.utils.get_file('mmod_human_face_detector.dat.bz2',
                            cache_subdir=os.path.abspath('.'),
                            origin = "http://dlib.net/files/mmod_human_face_detector.dat.bz2")
# Using pythons bz2 package to read the bz2 file in binary format and write it into a .dat file
with bz2.open("mmod_human_face_detector.dat.bz2", "rb") as f:
    content = f.read()

    with open("mmod_human_face_detector.dat", "wb") as weights_file:
        weights_file.write(content)

os.remove(annotation_zip)

694709/694709 [==============================] - 1s 1us/step


In [ ]:
dataset_url = 'https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar'
annotation_folder = "wiki_crop"
if not os.path.exists(os.path.abspath('.') + annotation_folder):
    annotation_zip = tf.keras.utils.get_file('wiki.tar',
                                            cache_subdir=os.path.abspath('.'),
                                            origin = dataset_url,
                                            extract = True)
    os.remove(annotation_zip)
data_key = 'wiki'
mat_file = 'wiki.mat'

811315200/811315200 [==============================] - 9s 0us/step


In [ ]:
dataset_url = 'https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar'
annotation_folder = "imdb_crop"
if not os.path.exists(os.path.abspath('.') + annotation_folder):
    annotation_zip = tf.keras.utils.get_file('imdb.tar',
                                            cache_subdir=os.path.abspath('.'),
                                            origin = dataset_url,
                                            extract = True)
    os.remove(annotation_zip)
data_key = 'imdb'
mat_file = 'imdb.mat'

7012157440/7012157440 [==============================] - 101s 0us/step


In [ ]:
!python mat.py

In [ ]:
!python gender.py

In [ ]:
# Importing dependencies
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Setting up the data generators
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('dataset/gender/train', target_size=(128, 128), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory('dataset/gender/test', target_size=(128, 128), batch_size=32, class_mode='binary')

# Building the model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)



# Compiling the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(train_generator, epochs=10, validation_data=test_generator)

# Saving the model
model.save('gender_model.h5')

Found 171100 images belonging to 2 classes.
Found 53740 images belonging to 2 classes.


Epoch 1/10
5347/5347 [==============================] - 1199s 217ms/step - loss: 0.4483 - accuracy: 0.7992 - val_loss: 0.6528 - val_accuracy: 0.7629
Epoch 2/10
5347/5347 [==============================] - 1180s 221ms/step - loss: 0.4103 - accuracy: 0.8189 - val_loss: 0.4970 - val_accuracy: 0.8337
Epoch 3/10
5347/5347 [==============================] - 1091s 204ms/step - loss: 0.3787 - accuracy: 0.8461 - val_loss: 0.4593 - val_accuracy: 0.8063
Epoch 4/10
5347/5347 [==============================] - 1100s 206ms/step - loss: 0.3547 - accuracy: 0.8600 - val_loss: 0.2545 - val_accuracy: 0.9195
Epoch 5/10
5347/5347 [==============================] - 1136s 212ms/step - loss: 0.3337 - accuracy: 0.8698 - val_loss: 0.3909 - val_accuracy: 0.8329
Epoch 6/10
5347/5347 [==============================] - 1156s 216ms/step - loss: 0.3228 - accuracy: 0.8751 - val_loss: 0.2758 - val_accuracy: 0.9048
Epoch 7/10
5347/5347 [==============================] - 1189s 222ms/step - loss: 0.3149 - accuracy: 0.8788

In [ ]:
# Evaluating the model on test data
score = model.evaluate(test_generator)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


1680/1680 [==============================] - 81s 48ms/step - loss: 0.2952 - accuracy: 0.8990
Test loss: 0.295164555311203
Test accuracy: 0.8989951610565186
